In [53]:
import sys
sys.path.append('../')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import SubsetRandomSampler
import numpy as np
import random

import numpy as np
import pandas as pd
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import sklearn

eps = np.finfo(float).eps

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
batch_size = 4
feat_dim = 512
tau = 1
x_i = torch.randn(batch_size, feat_dim)
x_j = x_i + 0.05 * torch.randn(batch_size, feat_dim)
x = torch.cat((x_i, x_j), dim=0)

In [55]:
sim_mat_nom = torch.mm(x, x.T)
sim_mat_denom = torch.mm(torch.norm(x, dim=1).unsqueeze(1), torch.norm(x, dim=1).unsqueeze(1).T)

sim_mat = sim_mat_nom / sim_mat_denom.clamp(min=1e-16)
sim_mat = torch.exp(sim_mat / tau)

print(sim_mat.size())

torch.Size([8, 8])


In [56]:
print(sim_mat)

tensor([[2.7183, 0.9925, 0.9922, 0.9922, 2.7149, 0.9885, 0.9907, 0.9878],
        [0.9925, 2.7183, 0.9610, 0.9498, 0.9938, 2.7145, 0.9618, 0.9467],
        [0.9922, 0.9610, 2.7183, 1.0612, 0.9920, 0.9635, 2.7149, 1.0597],
        [0.9922, 0.9498, 1.0612, 2.7183, 0.9880, 0.9476, 1.0629, 2.7151],
        [2.7149, 0.9938, 0.9920, 0.9880, 2.7183, 0.9896, 0.9906, 0.9836],
        [0.9885, 2.7145, 0.9635, 0.9476, 0.9896, 2.7183, 0.9645, 0.9446],
        [0.9907, 0.9618, 2.7149, 1.0629, 0.9906, 0.9645, 2.7183, 1.0614],
        [0.9878, 0.9467, 1.0597, 2.7151, 0.9836, 0.9446, 1.0614, 2.7183]])


In [57]:
i_ind = torch.arange(0, batch_size)
j_ind = torch.arange(batch_size, batch_size * 2)

diag_ind = torch.eye(batch_size * 2).bool()
sim_mat = sim_mat.masked_fill_(diag_ind, 0)

left_score = []
right_score = []
losses = []
for i in range(4):
    j = i + 4
    print('{}-{}'.format(i,j))
    loss = 2 * sim_mat[i][j] / torch.sum(sim_mat[i, :])
    losses.append(loss)

torch.mean(torch.tensor(losses))

0-4
1-5
2-6
3-7


tensor(0.6271)

In [59]:
class contrastive_loss(nn.Module):
    def __init__(self, tau=1):
        self.tau = tau

    def forwrad(self, x):
        b_sz = x.size(0) // 2

        sim_mat_nom = torch.mm(x, x.T)
        sim_mat_denom = torch.mm(torch.norm(x, dim=1).unsqueeze(1), torch.norm(x, dim=1).unsqueeze(1).T)

        sim_mat = sim_mat_nom / sim_mat_denom.clamp(min=1e-16)
        sim_mat = torch.exp(sim_mat / self.tau)

        # getting rid of diag
        diag_ind = torch.eye(b_sz * 2).bool()
        sim_mat = sim_mat.masked_fill_(diag_ind, 0)

        loss_nom = torch.zeros(b_sz)
        loss_denom = torch.zeros(b_sz)

        for i in range(b_sz):
            loss_nom[i] = sim_mat[i][i + b_sz]
            loss_denom[i] = torch.sum(sim_mat[i, :])

        loss = torch.mean(-torch.log(loss_nom / loss_denom))

        return loss

    